In [1]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars

In [2]:
include("contractions.jl")
include("visualisation.jl")

visualize (generic function with 2 methods)

In [9]:

function findPointInRadius(vertex, vertexSet, radius)
    for v in vertexSet
        if norm(vertex - v) < radius
            return v
        end
    end
    return nothing
end

function CreateSimplicialComplex2D(mesh)
    coords = Vector{Vector{Float32}}()
    anticoords = Dict{Vector{Float32},Int}()

    triangles_ = Vector{Tuple{Int,Int,Int}}()

    for triangle in ProgressBar(mesh)
        for vertex in triangle
            arr_vertex = collect(vertex)
            close_point = findPointInRadius(arr_vertex, coords, 0.0001)

            if close_point == nothing
                push!(coords, arr_vertex)
                anticoords[arr_vertex] = length(coords)
            else
                anticoords[arr_vertex] = anticoords[close_point]
            end
        end
        push!(triangles_, (anticoords[collect(triangle[1])], anticoords[collect(triangle[2])], anticoords[collect(triangle[3])]))
    end

    SimplicialComplex2D(triangles_, coords)
end

function loadsc(path)
    data = load(path)
    data_triangles = Array([Array([Tuple([round(z, digits=5) for z in y]) for y in x]) for x in data])
    CreateSimplicialComplex2D(data_triangles)
end

loadsc (generic function with 1 method)

In [10]:
bunidata = loadsc("bunny/reconstruction/bun_zipper.ply")
# fuzz = 1
# for i in 1:length(bunidata.coords)
#     bunidata.coords[i][1] *= rand() * fuzz
#     bunidata.coords[i][2] *= rand() * fuzz
#     bunidata.coords[i][3] *= rand() * fuzz
# end

0.0%┣                                          ┫ 0/69.5k [00:00<-27:-24, -0s/it]
0.0%┣                                        ┫ 1/69.5k [00:01<Inf:Inf, InfGs/it]
0.3%┣▏                                        ┫ 219/69.5k [00:01<07:17, 159it/s]
0.5%┣▏                                        ┫ 326/69.5k [00:01<05:03, 228it/s]
0.6%┣▎                                        ┫ 396/69.5k [00:01<04:18, 267it/s]
0.7%┣▎                                        ┫ 464/69.5k [00:02<03:48, 303it/s]
0.7%┣▎                                        ┫ 513/69.5k [00:02<03:33, 324it/s]
0.8%┣▎                                        ┫ 564/69.5k [00:02<03:20, 345it/s]
0.9%┣▍                                        ┫ 603/69.5k [00:02<03:12, 358it/s]
0.9%┣▍                                        ┫ 653/69.5k [00:02<03:03, 376it/s]
1.0%┣▍                                        ┫ 686/69.5k [00:02<02:59, 384it/s]
1.0%┣▍                                        ┫ 726/69.5k [00:02<02:54, 395it/s]
1.1%┣▌                      

SimplicialComplex2D(Vector{Float32}[[-0.09231, 0.13236, 0.01822], [-0.09218, 0.13235, 0.01722], [-0.09277, 0.13099, 0.01723], [-0.08717, 0.11039, 0.01836], [-0.08896, 0.11186, 0.01834], [-0.08744, 0.11045, 0.01734], [-0.08784, 0.10502, 0.01437], [-0.08724, 0.10636, 0.01436], [-0.08725, 0.10635, 0.01336], [-0.08785, 0.10502, 0.01337]  …  [-0.0419, 0.03367, 0.02694], [-0.0428, 0.03443, 0.02962], [-0.04345, 0.03445, 0.03123], [-0.04408, 0.03448, 0.03283], [-0.04464, 0.03465, 0.03432], [-0.04532, 0.03459, 0.03598], [-0.04592, 0.03469, 0.03752], [-0.04664, 0.03455, 0.03923], [-0.04724, 0.03464, 0.04077], [-0.04787, 0.03465, 0.04236]], Set([24824, 25754, 29965, 11950, 31794, 30270, 1703, 12427, 7685, 18374  …  4724, 4496, 10808, 8546, 4159, 8992, 15380, 17984, 19700, 21664]), Dict{Int64, Set{Tuple{Int64, Int64}}}(24824 => Set([(24824, 24893), (24824, 24895), (24824, 24894), (24760, 24824), (24824, 24825), (24759, 24824), (24823, 24824)]), 25754 => Set([(2462, 25754), (25753, 25754), (25754, 

In [14]:
using JLD2

In [15]:
save_object("bunidata.jld2", bunidata)

In [16]:
using Serialization

In [17]:
serialize("bunidata.dat", bunidata)

In [21]:
using JSON3

In [22]:
JSON3.write("bunidata.json", bunidata)

"bunidata.json"

In [23]:
buni = initialContractedSimplicialComplex2D(bunidata)

ContractedSimplicialComplex2D(SimplicialComplex2D(Vector{Float32}[[-0.09231, 0.13236, 0.01822], [-0.09218, 0.13235, 0.01722], [-0.09277, 0.13099, 0.01723], [-0.08717, 0.11039, 0.01836], [-0.08896, 0.11186, 0.01834], [-0.08744, 0.11045, 0.01734], [-0.08784, 0.10502, 0.01437], [-0.08724, 0.10636, 0.01436], [-0.08725, 0.10635, 0.01336], [-0.08785, 0.10502, 0.01337]  …  [-0.0419, 0.03367, 0.02694], [-0.0428, 0.03443, 0.02962], [-0.04345, 0.03445, 0.03123], [-0.04408, 0.03448, 0.03283], [-0.04464, 0.03465, 0.03432], [-0.04532, 0.03459, 0.03598], [-0.04592, 0.03469, 0.03752], [-0.04664, 0.03455, 0.03923], [-0.04724, 0.03464, 0.04077], [-0.04787, 0.03465, 0.04236]], Set([24824, 25754, 29965, 11950, 31794, 30270, 1703, 12427, 7685, 18374  …  4724, 4496, 10808, 8546, 4159, 8992, 15380, 17984, 19700, 21664]), Dict{Int64, Set{Tuple{Int64, Int64}}}(24824 => Set([(24824, 24893), (24824, 24895), (24824, 24894), (24760, 24824), (24824, 24825), (24759, 24824), (24823, 24824)]), 25754 => Set([(2462, 25

In [26]:
println(" - edge has 2 adjacent triangles")
is_missing = false
marked_vertices = Set{Int}()
for edge in edges(buni.original)
    if length(buni.original._edge_to_triangles[edge]) != 2
        is_missing = true
        for vertex in edge
            push!(marked_vertices, vertex)
        end
    end
end
println("result: ", !is_missing)
println("marked vertices: ", length(marked_vertices))

In [28]:
# convert to GeometryBasics.Point{3,Float32}
marks::Vector{GeometryBasics.Point{3,Float32}} = [GeometryBasics.Point{3,Float32}(buni.original.coords[x]) for x in marked_vertices]
visualize(buni, [marks...])

GLMakie.Screen(...)

In [7]:
pq = PriorityQueue()
for (edge, triangle) in ProgressBar(buni.contracted._edge_to_triangles)
    pq[edge] = error(buni, edge)
end

0.0%┣                                   ┫ 0/208.4k [00:14<-813:-51:-37, -14s/it]
0.0%┣                                       ┫ 1/208.4k [00:25<Inf:Inf, InfGs/it]
0.0%┣                                       ┫ 11/208.4k [00:25<145:35:05, 3s/it]
1.8%┣▊                                      ┫ 3.7k/208.4k [00:25<22:59, 148it/s]
3.5%┣█▍                                     ┫ 7.3k/208.4k [00:25<11:37, 289it/s]
5.2%┣██                                    ┫ 10.9k/208.4k [00:25<07:38, 432it/s]
7.1%┣██▊                                   ┫ 14.7k/208.4k [00:25<05:34, 580it/s]
8.8%┣███▍                                  ┫ 18.4k/208.4k [00:25<04:23, 723it/s]
10.5%┣███▉                                 ┫ 21.8k/208.4k [00:25<03:38, 855it/s]
12.5%┣████▌                               ┫ 26.0k/208.4k [00:26<02:59, 1.0kit/s]
14.3%┣█████▏                              ┫ 29.8k/208.4k [00:26<02:33, 1.2kit/s]
16.0%┣█████▊                              ┫ 33.4k/208.4k [00:26<02:14, 1.3kit/s]
17.8%┣██████▍               

In [8]:
for i in ProgressBar(1:3)
    first = dequeue!(pq)
    while !issafe(buni.contracted, first)
        first = dequeue!(pq)
    end

    new_vertex = contract!(buni, first)
    for edge in buni.contracted._vertex_to_edges[new_vertex]
        pq[edge] = error(buni, edge)
        for triangle in buni.contracted._edge_to_triangles[edge]
            pq[triangle] = error(buni, triangle)
        end
    end
end

0.0%┣                                                ┫ 0/3 [00:00<00:00, -0s/it]


BoundsError: BoundsError: attempt to access Set{Int64} with 1 element at index [2]